In [ ]:
import os
print(os.listdir("../input"))
print(os.getcwd()) #current working directory
# Any results you write to the current directory are saved as output.

**CASE 1 : ANALYSIS ON THE SIZE OF THE VOTERS, TURN OUT & SPOILT VOTES DURING GE-14**

Using provided data , the following sub-cases to be observed :

1. Parliament seat vs voters size analysis
2. State seat vs voters size analysis
3. Turn out analysis
4. Spoilt votes analysis


In [ ]:
import pandas as pd

#case1
#reading data for total votes by seat
vseat = pd.read_csv("../input/Election-Results-2018 - Total_Votes_by_Seat.csv")
#table info 
vseat.info()
#view table first 10 row
vseat.head(10)

In [ ]:
#describe table for case1
vseat.describe()

In [ ]:
#data cleaning for case1
#assign string to integer
vseat["Votes for ALL Candidate"] = vseat["Votes for ALL Candidate"].str.replace(",","").astype(int)
vseat["Spoilt Votes"] = vseat["Spoilt Votes"].str.replace(",","").astype(int)
vseat["Total Registered Votes"] = vseat["Total Registered Votes"].str.replace(",","").astype(int)

#assign string to float
vseat["Turn Out"] = vseat["Turn Out"].str.replace("%","").astype(float)/100.

#rename column
vseat.rename(columns = {'Turn Out':'Turn Out %'}, inplace = True)
vseat['Turn Out %'] = vseat['Turn Out %'].map(lambda x: '%2.2f' % x).astype(float)

#create new colum on numerical operation
vseat["Turn Out"]=(vseat['Total Registered Votes']*vseat["Turn Out %"]).astype(int)

#create new colum on numerical operation
vseat["No Turn Out %"]=(1-vseat["Turn Out %"]).map(lambda x: '%2.2f' % x).astype(float)
vseat["No Turn Out"]=(vseat['Total Registered Votes']-vseat["Votes for ALL Candidate"]).astype(int)

#uppercase
vseat['State'] = vseat['State'].str.upper()

#create new colum on numerical operation
#replace NaN values
vseat["Spoilt Votes %"]=(vseat["Spoilt Votes"]/vseat["Votes for ALL Candidate"]).map(lambda x: '%2.2f' % x).astype(float)
vseat = vseat.fillna(0)
vseat.info()


In [ ]:
#count no of Parliment & DUN seat
print('Number of Parliament Seat :',vseat[(vseat.Type == 'PARLIMEN')].count()['Seat'])
print('Number of DUN Seat :',vseat[(vseat.Type == 'DUN')].count()['Seat'])


In [ ]:
#df for par and state seat
vseatDUN = vseat[vseat.Type=='DUN'].reset_index()
vseatPAR = vseat[vseat.Type=='PARLIMEN'].reset_index()

#confirm table size (rows, features)
#print(vseat.shape)
#print(vseatDUN.shape)
#print(vseatPAR.shape)


In [ ]:
import numpy as np

#PAR and DUN seat aggregation by state 

#joining group df
vseat_AGGst=vseat[vseat.Type=='PARLIMEN'].groupby(['State']).size().reset_index(name='Total_PAR').\
merge((vseat[vseat.Type=='DUN'].groupby(['State']).size().reset_index(name='Total_DUN')),how='left').\
merge((vseat[vseat.Type=='PARLIMEN'].groupby(['State']).sum()['Total Registered Votes'].reset_index(name="Total_Voters"))).\
merge((vseat[vseat.Type=='PARLIMEN'].groupby(['State']).sum()['Turn Out'].reset_index(name="Total_Turn_Out_PAR"))).\
merge((vseat[vseat.Type=='DUN'].groupby(['State']).sum()['Turn Out'].reset_index(name="Total_Turn_Out_DUN")),how='left').\
merge((vseat[vseat.Type=='PARLIMEN'].groupby(['State']).sum()['Spoilt Votes'].reset_index(name="Total_Spoilt_Votes_PAR"))).\
merge((vseat[vseat.Type=='DUN'].groupby(['State']).sum()['Spoilt Votes'].reset_index(name="Total_Spoilt_Votes_DUN")),how='left').\
merge((vseat[vseat.Type=='PARLIMEN'].groupby(['State']).sum()['Votes for ALL Candidate'].reset_index(name="Total_Votes_PAR"))).\
merge((vseat[vseat.Type=='DUN'].groupby(['State']).sum()['Votes for ALL Candidate'].reset_index(name="Total_Votes_DUN")),how='left')

#calculate percentage
vseat_AGGst['Total_Voters_PCT']=(vseat_AGGst['Total_Voters']/vseatPAR.sum()['Total Registered Votes']).map(lambda x: '%2.2f' % x).astype(float)
vseat_AGGst['Turn_Out_PCT_PAR']=(vseat_AGGst['Total_Turn_Out_PAR']/vseat_AGGst['Total_Voters']).map(lambda x: '%2.2f' % x).astype(float)
vseat_AGGst['Turn_Out_PCT_DUN']=(vseat_AGGst['Total_Turn_Out_DUN']/vseat_AGGst['Total_Voters']).map(lambda x: '%2.2f' % x).astype(float)
vseat_AGGst['Spoilt_Votes_PCT_PAR']=(vseat_AGGst['Total_Spoilt_Votes_PAR']/vseat_AGGst['Total_Turn_Out_PAR']).map(lambda x: '%2.2f' % x).astype(float)
vseat_AGGst['Spoilt_Votes_PCT_DUN']=(vseat_AGGst['Total_Spoilt_Votes_DUN']/vseat_AGGst['Total_Turn_Out_DUN']).map(lambda x: '%2.2f' % x).astype(float)
vseat_AGGst['Total_Votes_PCT_PAR']=(vseat_AGGst['Total_Votes_PAR']/vseat_AGGst['Total_Turn_Out_PAR']).map(lambda x: '%2.2f' % x).astype(float)
vseat_AGGst['Total_Votes_PCT_DUN']=(vseat_AGGst['Total_Votes_DUN']/vseat_AGGst['Total_Turn_Out_DUN']).map(lambda x: '%2.2f' % x).astype(float)

#built df
vseat_AGGst[['State','Total_PAR','Total_DUN','Total_Voters','Total_Voters_PCT','Turn_Out_PCT_PAR','Turn_Out_PCT_DUN','Spoilt_Votes_PCT_PAR','Spoilt_Votes_PCT_DUN',
            'Total_Votes_PCT_PAR','Total_Votes_PCT_DUN']]

#calculate voter to parliament ratio
vseat_AGGst['Voters_to_Par_Ratio']=(vseat_AGGst['Total_Voters']/vseat_AGGst['Total_PAR']).round().astype(int)

#group by ratio size
vseat_AGGst['Voters_to_Par_Ratio_Class']=np.where(vseat_AGGst['Voters_to_Par_Ratio'] >= 80000,'HIGH RATIO'
                                                  ,np.where( (vseat_AGGst['Voters_to_Par_Ratio'] < 80000) & (vseat_AGGst['Voters_to_Par_Ratio'] > 50000),'MODERATE RATIO','LOW RATIO'))

#group by dominancy                                                                                                                          
vseat_AGGst['Dominancy']=np.where(vseat_AGGst['Total_PAR']/222 >= 0.1,'DOMINANT',np.where(vseat_AGGst['Total_PAR']/222 <= 0.05,'LESS DOMINANT','SEMI DOMINANT'))


 <br>
 <font size="+2">**Sub Case 1 : Parliament Seat vs Voters Size Analysis**</font>

In [ ]:
import matplotlib.ticker as ticker
vseat_AGGst.sort_values('Total_PAR').plot.barh(title='TOTAL PARLIAMENT/DUN SEAT BY STATE',legend=False,x='State',y=['Total_PAR','Total_DUN'],
                                               figsize=(15,5)).xaxis.set_major_locator(ticker.MultipleLocator(1))

The above chart indicate numbers of parliamentary seat and DUN for each of the state in Malaysia . 
<br><br>
Note:
<br><br>
-DUN seat for Sarawak not indicated as the state election done saperately
<br>-There no DUN seat for Federal Territory (Wilayah Persekutuan)

In [ ]:
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(15,15))

plt.subplot(1,2,1)
vseatPAR.groupby(vseatPAR['State']).count()['Seat'].plot.pie(title='PERCENTAGE OF PARLIAMENT SEAT',autopct='%1.1f%%',fontsize=8).title.set_size(8)
plt.subplot(1,2,2)
vseatPAR.groupby(vseatPAR['State']).sum()['Total Registered Votes'].plot.pie(title='PERCENTAGE OF PARLIAMENT SEAT VOTERS',autopct='%1.1f%%',fontsize=8).title.set_size(8)


As indicate in the pie chart, the state of Sarawak, Sabah, Perak, Johor & Selangor have approximately more than 10% each from 222 parliamentary seats. These state can be described as 'Dominant' states
<br><br>
States which have approximately less than 5% from 222 parliamentary seats are Wilayah Persekutuan (Putrajaya & Labuan), Perlis, Terengganu, Melaka and Negeri Sembilan. These are 'Less-Dominant' states
<br><br>
Rest of the state (Pulau Pinang, Kedah, Pahang, Kelantan and W.P KL) which is  between 5% - 10% from total parliamentary seat can be described as 'Semi-Dominant' state.


In [ ]:
vseat_AGGst[['State','Total_PAR','Total_DUN','Total_Voters','Dominancy','Voters_to_Par_Ratio','Voters_to_Par_Ratio_Class']]

The following table describe total number of seat for each states and numbers of voters. Dominancy of each state categorized as per above definition  . I had also calculate ratio of voters to parliament seat for each state

In [ ]:
fig=plt.figure(figsize=(15,5))
(vseatPAR.groupby(vseatPAR['State']).sum()['Total Registered Votes'] / vseatPAR.groupby(vseatPAR['State']).count()['Seat']).\
sort_values().plot.bar(title='RATIO OF TOTAL VOTERS TO TOTAL PARLIAMENT SEAT',fontsize=8,color='blue').title.set_size(8)

The bar chart indicates ratio of total voters to total numbers of parliamentary seat allocated to the states. 
<br><br>States that have high ratio of  voters to parliament (more than 80,000) are Selangor, Terengganu & Melaka and tagged as 'High Ratio' states. 
<br><br>Perak, Johor, Pulau Pinang, Kelantan, Kedah,Pahang, W.P.KL  & Negeri Sembilan which have ratio between 50,000 to 80,000 tagged as 'Moderate Ratio' states.
<br><br>Rest of the state Perlis, WP Labuan, WP Putrajaya, Sabah & Sarawak which have ratio less than 50,000 tagged as 'Low Ratio' states  .

In [ ]:
vseat_AGGst.groupby(['State', 'Voters_to_Par_Ratio_Class']).size().reset_index().pivot(columns='State', 
                                                                       index='Voters_to_Par_Ratio_Class', values=0).plot(kind='barh', 
                                                                                                         stacked=True,figsize=(15, 6),
                                                                                                         color=['#3182bd','#6baed6','#9ecae1','#c6dbef','#e6550d','#fd8d3c','#fdae6b',
                                                                                                               '#fdd0a2','#31a354','#74c476','#a1d99b','#c7e9c0','#756bb1','#9e9ac8',
                                                                                                               '#bcbddc','#dadaeb'])

These ratio categories reflects on the effort required from the political entities to plan on the prioritizing in term of budgetories, manpower,logistics etc.

In [ ]:
vseat_AGGst.groupby(['Dominancy', 'Voters_to_Par_Ratio_Class']).size().reset_index().pivot(columns='Voters_to_Par_Ratio_Class', 
                                                                       index='Dominancy', values=0).plot(kind='barh', 
                                                                                                         stacked=True,figsize=(15, 6),
                                                                                                         color=['#3182bd','#6baed6','#9ecae1','#c6dbef','#e6550d','#fd8d3c','#fdae6b',
                                                                                                               '#fdd0a2','#31a354','#74c476','#a1d99b','#c7e9c0','#756bb1','#9e9ac8',
                                                                                                               '#bcbddc','#dadaeb'])

It is obvious that, for any political entities to dominate parliamentary seat to rule the government they should be focusing more on 'Dominant' and 'Semi-Dominant' state regardless volume of the voters
<br><br>
In term of political planning on budgetory and mobilization, 3 main factor which always been considered are :-
<br><br> a. Dominancy (higher dominancy means higher prioritization)
<br>b. Voters Density (higher density means higher prioritization)
<br>c. (a) overrule (b)

Taking the above factor into account, 4 states with the 'Dominant' status and 5 states with 'Semi-Dominant' status should be prioritized in order to secure numbers in parliamentary seats.
<br><br>
7 other states with 'Less Dominant' status to be prioritized according to the voters density. 

In [ ]:
vseat_AGGst.groupby(['Dominancy', 'State']).size().reset_index().pivot(columns='State', 
                                                                       index='Dominancy', values=0).plot(kind='barh', 
                                                                                                         stacked=True,figsize=(15, 6),
                                                                                                         color=['#3182bd','#6baed6','#9ecae1','#c6dbef','#e6550d','#fd8d3c','#fdae6b',
                                                                                                               '#fdd0a2','#31a354','#74c476','#a1d99b','#c7e9c0','#756bb1','#9e9ac8',
                                                                                                               '#bcbddc','#dadaeb'])

The above bar plots indicate that 'Less Dominant' status state have the highest count of state compared to 'Dominant' status state which have the lowest count.  The following bar plots explained why it is. 

In [ ]:
vseat_AGGst.groupby(['Dominancy']).sum()['Total_PAR'].plot.barh(figsize=(15, 6))

To win election and took over the administrative in Malaysia , political party need to win at 112 parliament saet. 
<br><br>The above bar chart self-explain how the 'dominancy' classification derived. 
<br><br>If any political party could manage to win MAJORITY seat in ALL 'Dominant' status state (Johor,Perak, Sabah & Sarawak) and win some at 'Semi-Dominant' and 'Less Dominant' state, they would likely win the election. 

 <br>
 <font size="+2">**Sub Case 2 : State Seat vs Voters Size Analysis**</font>

In [ ]:
import squarify

fig = plt.figure(figsize=(25, 6))
plt.title('TOTAL VOTERS BY STATE')
squarify.plot(sizes=vseat_AGGst['Total_Voters'], label=vseat_AGGst['State'] + ": " +vseat_AGGst['Total_Voters'].astype('str'), alpha=.8)
plt.axis('off')
plt.show()

fig = plt.figure(figsize=(25, 6))
plt.title('TOTAL PARLIAMENTARY SEAT BY STATE')
squarify.plot(sizes=vseat_AGGst['Total_PAR'], label=vseat_AGGst['State'] + ": " +vseat_AGGst['Total_PAR'].astype('str'), alpha=.8)
plt.axis('off')
plt.show()

fig = plt.figure(figsize=(25, 6))
plt.title('VOTER TO PARLIAMENT RATIO BY STATE')
squarify.plot(sizes=vseat_AGGst['Voters_to_Par_Ratio'], label=vseat_AGGst['State'] + ": " +vseat_AGGst['Voters_to_Par_Ratio'].astype('str'), alpha=.8)
plt.axis('off')
plt.show()

 

In [ ]:
fig, axarr = plt.subplots(3,2,squeeze=False,figsize=(20,10))

vseat_AGGst.sort_values('Turn_Out_PCT_PAR').plot.barh(title='Turn Out % Parliament Seat',legend=False,x='State',y=['Turn_Out_PCT_PAR'],ax =axarr[0][0])
vseat_AGGst.sort_values('Turn_Out_PCT_DUN').plot.barh(title='Turn Out % State Seat',legend=False,x='State',y=['Turn_Out_PCT_DUN'],ax =axarr[0][1])
vseat_AGGst.sort_values('Total_Votes_PCT_PAR').plot.barh(title='Total Votes % Parliament Seat',legend=False,x='State',y=['Total_Votes_PCT_PAR'],ax =axarr[1][0])
vseat_AGGst.sort_values('Total_Votes_PCT_DUN').plot.barh(title='Total Votes % State Seat',legend=False,x='State',y=['Total_Votes_PCT_DUN'],ax =axarr[1][1])
vseat_AGGst.sort_values('Spoilt_Votes_PCT_PAR').plot.barh(x='State',y=['Spoilt_Votes_PCT_PAR'],ax =axarr[2][0])
vseat_AGGst.sort_values('Spoilt_Votes_PCT_DUN').plot.barh(x='State',y=['Spoilt_Votes_PCT_DUN'],ax =axarr[2][1])


In [ ]:
fig = plt.figure(figsize=(25, 6))
plt.title('Voters Size by State Seat in Perlis')
squarify.plot(sizes=vseat[vseat['State']=='PERLIS']['Total Registered Votes'],label=vseat[vseat['State']=='PERLIS']['Seat'], alpha=.9 )
plt.axis('off')
plt.show()

fig = plt.figure(figsize=(25, 6))
plt.title('Voters Size by State Seat in Penang')
squarify.plot(sizes=vseat[vseat['State']=='PULAU PINANG']['Total Registered Votes'],label=vseat[vseat['State']=='PULAU PINANG']['Seat'], alpha=.9 )
plt.axis('off')
plt.show()

fig = plt.figure(figsize=(25, 6))
plt.title('Voters Size by State Seat in Kedah')
squarify.plot(sizes=vseat[vseat['State']=='KEDAH']['Total Registered Votes'],label=vseat[vseat['State']=='KEDAH']['Seat'], alpha=.9 )
plt.axis('off')
plt.show()


In [ ]:
fig = plt.figure(figsize=(25, 6))
plt.title('Voters Size by State Seat in Melaka')
squarify.plot(sizes=vseat[vseat['State']=='MELAKA']['Total Registered Votes'],label=vseat[vseat['State']=='MELAKA']['Seat'], alpha=.9 )
plt.axis('off')
plt.show()

fig = plt.figure(figsize=(25, 6))
plt.title('Voters Size by State Seat in Johor')
squarify.plot(sizes=vseat[vseat['State']=='JOHOR']['Total Registered Votes'],label=vseat[vseat['State']=='JOHOR']['Seat'], alpha=.9 )
plt.axis('off')
plt.show()

In [ ]:
fig = plt.figure(figsize=(25, 6))
plt.title('Voters Size by State Seat in Perak')
squarify.plot(sizes=vseat[vseat['State']=='PERAK']['Total Registered Votes'],label=vseat[vseat['State']=='PERAK']['Seat'], alpha=.9 )
plt.axis('off')
plt.show()

fig = plt.figure(figsize=(25, 6))
plt.title('Voters Size by State Seat in Selangor')
squarify.plot(sizes=vseat[vseat['State']=='SELANGOR']['Total Registered Votes'],label=vseat[vseat['State']=='SELANGOR']['Seat'], alpha=.9 )
plt.axis('off')
plt.show()

fig = plt.figure(figsize=(25, 6))
plt.title('Voters Size by State Seat in Negeri Sembilan')
squarify.plot(sizes=vseat[vseat['State']=='NEGERI SEMBILAN']['Total Registered Votes'],label=vseat[vseat['State']=='NEGERI SEMBILAN']['Seat'], alpha=.9 )
plt.axis('off')
plt.show()

In [ ]:
fig, axarr = plt.subplots(4,2,squeeze=False,figsize=(40,50))

vseat[(vseat.Type == 'PARLIMEN') & (vseat.State == 'SELANGOR')].sort_values('Total Registered Votes').plot.bar(title='Selangor Parliament', x='Seat',y='Total Registered Votes',color='red',ax =axarr[0][0])
vseat[(vseat.Type == 'DUN') & (vseat.State == 'SELANGOR')].sort_values('Total Registered Votes').plot.barh(title='Selangor DUN',x='Seat',y='Total Registered Votes',color='red',ax =axarr[0][1])
vseat[(vseat.Type == 'PARLIMEN') & (vseat.State == 'PULAU PINANG')].sort_values('Total Registered Votes').plot.bar(x='Seat',y='Total Registered Votes',color='red',ax =axarr[1][0])
vseat[(vseat.Type == 'DUN') & (vseat.State == 'PULAU PINANG')].sort_values('Total Registered Votes').plot.barh(x='Seat',y='Total Registered Votes',color='red',ax =axarr[1][1])
vseat[(vseat.Type == 'PARLIMEN') & (vseat.State == 'PERLIS')].sort_values('Total Registered Votes').plot.bar(x='Seat',y='Total Registered Votes',color='blue',ax =axarr[2][0])
vseat[(vseat.Type == 'DUN') & (vseat.State == 'PERLIS')].sort_values('Total Registered Votes').plot.barh(x='Seat',y='Total Registered Votes',color='blue',ax =axarr[2][1])
vseat[(vseat.Type == 'PARLIMEN') & (vseat.State == 'KEDAH')].sort_values('Total Registered Votes').plot.bar(x='Seat',y='Total Registered Votes',color='red',ax =axarr[3][0])
vseat[(vseat.Type == 'DUN') & (vseat.State == 'KEDAH')].sort_values('Total Registered Votes').plot.barh(x='Seat',y='Total Registered Votes',color='red',ax =axarr[3][1])


In [ ]:
#plot turn out % vs state,par and DUN
#plot side by side (1x2)

fig, axarr = plt.subplots(1, 2, figsize=(20, 9))

vseat[(vseat.Type == 'PARLIMEN') & (vseat.State == 'SELANGOR')].groupby('Seat').agg(['sum']
                                                                                ).plot.barh(y='No Turn Out %',ax=axarr[0],legend='loc=2',color="green",
                                                                                                    fontsize=8,title='Malaysia GE-14 No Turn Out % by Parliament ')

vseat[(vseat.Type == 'DUN') & (vseat.State == 'SELANGOR')].groupby('Seat').agg(['sum']
                                                                                ).plot.barh(y='No Turn Out %',ax=axarr[1],legend='loc=2',color="green",
                                                                                                    fontsize=8,title='Malaysia GE-14 No Turn Out % by DUN ')



In [ ]:
#plot spoilt votes vs state,par and DUN
#plot side by side (1x2)
fig, axarr = plt.subplots(1, 2, figsize=(20, 4.5))


vseat[(vseat.Type == 'PARLIMEN') & (vseat.State == 'MELAKA')].groupby('Seat').agg(['sum']
                                                                                ).plot.barh(y='Spoilt Votes %',ax=axarr[0],legend='loc=2',color="red",
                                                                                                    fontsize=8,title='Malaysia GE-14 Spoilt Votes by Parliament ')

vseat[(vseat.Type == 'DUN') & (vseat.State == 'MELAKA')].groupby('Seat').agg(['sum']
                                                                                ).plot.barh(y='Spoilt Votes %',ax=axarr[1],legend='loc=2',color="red",
                                                                                                    fontsize=8,title='Malaysia GE-14 Spoilt Votes by DUN ')

In [ ]:
#case2
#reading data for parlimen result by candidate
vparc = pd.read_csv("../input/Election-Results-2018 - Parlimen_Results_By_Candidate.csv")
vparc.info()
vparc.head()

In [ ]:
#case3
#reading data for total votes by state
vsttc = pd.read_csv("../input/Election-Results-2018 - State_Results_By_Candidate.csv")
vsttc.info()
vsttc.head()

In [ ]:
## to be continue ..